In [ ]:
# ==========================================
# 1. INSTALL COMPATIBILITY PACKAGES
# ==========================================
!pip install -q tf-keras tensorflow-text

import os

# Force legacy Keras behavior
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"  

import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import kagglehub

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight

# Legacy Keras
import tf_keras as keras
from tf_keras.layers import Input, Dropout, Dense
from tf_keras.models import Model
from tf_keras.callbacks import EarlyStopping

# ==========================================
# 2. GPU VERIFICATION & CONFIG
# ==========================================
print("\n🔍 Checking GPU availability...")
gpus = tf.config.list_physical_devices('GPU')

if gpus:
    print(f"✅ GPU detected: {gpus}")
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    raise RuntimeError("❌ GPU NOT FOUND. Enable GPU in Kaggle settings.")

print("TensorFlow version:", tf.__version__)

# ==========================================
# 3. LOAD & PREPROCESS DATA
# ==========================================
print("\n📥 Downloading dataset...")
path = kagglehub.dataset_download("mayurjare/emotion-datasets")
csv_file = [f for f in os.listdir(path) if f.endswith('.csv')][0]
df = pd.read_csv(os.path.join(path, csv_file))

label_map = {
    'joy':'happy', 'sadness':'sad', 'anger':'anger',
    'fear':'fear', 'surprise':'surprise',
    'neutral':'neutral', 'disgust':'disgust'
}

df = df[df['Emotion'].isin(label_map.keys())]
df['Emotion'] = df['Emotion'].map(label_map)
df['Clean_Text'] = df['Text'].astype(str).str.strip()

le = LabelEncoder()
y_encoded = le.fit_transform(df['Emotion'])
y = keras.utils.to_categorical(y_encoded, num_classes=7)

X_train, X_test, y_train, y_test = train_test_split(
    df['Clean_Text'],
    y,
    test_size=0.2,
    random_state=42,
    stratify=y_encoded
)

# ==========================================
# 4. CLASS WEIGHTS
# ==========================================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_encoded),
    y=y_encoded
)
class_weight_dict = dict(enumerate(class_weights))
print("⚖️ Class Weights:", class_weight_dict)

# ==========================================
# 5. BUILD BERT MODEL
# ==========================================
bert_preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
bert_encoder_url = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/1"

def build_bert_model():
    text_input = Input(shape=(), dtype=tf.string, name='text')

    preprocessing = hub.KerasLayer(
        bert_preprocess_url, name='preprocessing'
    )
    encoder_inputs = preprocessing(text_input)

    encoder = hub.KerasLayer(
        bert_encoder_url,
        trainable=True,
        name='BERT_encoder'
    )

    outputs = encoder(encoder_inputs)

    x = outputs['pooled_output']
    x = Dropout(0.3)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(7, activation='softmax', name='classifier')(x)

    return Model(text_input, x)

with tf.device('/GPU:0'):
    model = build_bert_model()

model.summary()

# ==========================================
# 6. PHASE 1 – TRAIN CLASSIFIER HEAD
# ==========================================
for layer in model.layers:
    if "BERT_encoder" in layer.name:
        layer.trainable = False

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=3e-4),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

print("\n🚀 PHASE 1: Training classifier head (GPU)")
with tf.device('/GPU:0'):
    model.fit(
        X_train,
        y_train,
        epochs=2,
        batch_size=32,
        validation_data=(X_test, y_test)
    )

# ==========================================
# 7. PHASE 2 – FINE-TUNE BERT
# ==========================================
for layer in model.layers:
    if "BERT_encoder" in layer.name:
        layer.trainable = True

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=2e-5),
    loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
    metrics=['accuracy']
)

early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

print("\n🔥 PHASE 2: Fine-tuning BERT (GPU)")
with tf.device('/GPU:0'):
    history = model.fit(
        X_train,
        y_train,
        epochs=8,
        batch_size=32,
        validation_data=(X_test, y_test),
        callbacks=[early_stop],
        class_weight=class_weight_dict
    )

# ==========================================
# 8. EVALUATION
# ==========================================
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\n🎯 Final Accuracy: {accuracy*100:.2f}%")

# ==========================================
# 9. LIVE PREDICTION
# ==========================================
def predict_emotion(text_list):
    preds = model.predict(np.array(text_list))
    for i, text in enumerate(text_list):
        idx = np.argmax(preds[i])
        label = le.inverse_transform([idx])[0]
        conf = preds[i][idx]
        print(f"Input: {text}")
        print(f"Prediction: {label.upper()} | Confidence: {conf:.2f}\n")

print("\n🧪 LIVE DEMO")
predict_emotion([
    "I can't believe I finally passed the exam!",
    "The dark alleyway made my heart race.",
    "This is just a regular Tuesday."
])


2025-12-27 17:19:00.543561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766855940.776516      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766855940.852152      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766855941.434177      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766855941.434217      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766855941.434220      24 computation_placer.cc:177] computation placer alr


🔍 Checking GPU availability...
✅ GPU detected: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow version: 2.19.0

📥 Downloading dataset...
⚖️ Class Weights: {0: np.float64(1.1518335051032282), 1: np.float64(5.782042723631509), 2: np.float64(0.9148666490625825), 3: np.float64(0.448114854814719), 4: np.float64(2.1958423120801114), 5: np.float64(0.7363029710545331), 6: np.float64(1.2184708447633115)}


I0000 00:00:1766855956.433587      24 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 text (InputLayer)           [(None,)]                    0         []                            
                                                                                                  
 preprocessing (KerasLayer)  {'input_type_ids': (None,    0         ['text[0][0]']                
                             128),                                                                
                              'input_mask': (None, 128)                                           
                             , 'input_word_ids': (None,                                           
                              128)}                                                               
                                                                                              

I0000 00:00:1766855970.997201      70 service.cc:152] XLA service 0x7a7cacc31270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1766855970.997234      70 service.cc:160]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1766855971.097770      70 cuda_dnn.cc:529] Loaded cuDNN version 91002
I0000 00:00:1766855971.333965      70 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


867/867 [==============================] - 79s 83ms/step - loss: 1.6334 - accuracy: 0.4024 - val_loss: 1.4822 - val_accuracy: 0.4944
Epoch 2/2
867/867 [==============================] - 74s 86ms/step - loss: 1.5155 - accuracy: 0.4681 - val_loss: 1.4392 - val_accuracy: 0.5186

🔥 PHASE 2: Fine-tuning BERT (GPU)
Epoch 1/8
867/867 [==============================] - 198s 209ms/step - loss: 1.4266 - accuracy: 0.5045 - val_loss: 1.3356 - val_accuracy: 0.5618
Epoch 2/8
867/867 [==============================] - 168s 194ms/step - loss: 1.2408 - accuracy: 0.5936 - val_loss: 1.2604 - val_accuracy: 0.6110
Epoch 3/8
867/867 [==============================] - 168s 193ms/step - loss: 1.1225 - accuracy: 0.6502 - val_loss: 1.2233 - val_accuracy: 0.6283
Epoch 4/8
867/867 [==============================] - 167s 193ms/step - loss: 1.0349 - accuracy: 0.6902 - val_loss: 1.1847 - val_accuracy: 0.6638
Epoch 5/8
867/867 [==============================] - 167s 193ms/step - loss: 0.9505 - accuracy: 0.7297 - val_

In [ ]:
import pickle


model.save("emotion_bert_model", save_format="tf")

# Save LabelEncoder
with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)

print("✅ Model and LabelEncoder saved successfully")


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape(None), tf.float32, tf.int32, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c9a7d3350>, 134675615336688), {}).


INFO:tensorflow:Unsupported signature for serialization: ((IndexedSlicesSpec(TensorShape(None), tf.float32, tf.int32, tf.int32, TensorShape([None])), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c9a7d3350>, 134675615336688), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c9a2d3890>, 134675596513952), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c9a2d3890>, 134675596513952), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(2, 512), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c9a104b00>, 134675596514032), {}).


...

INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c99d07290>, 134675612972288), {}).


INFO:tensorflow:Unsupported signature for serialization: ((TensorSpec(shape=(512,), dtype=tf.float32, name='gradient'), <tensorflow.python.framework.func_graph.UnknownArgument object at 0x7a7c99d07290>, 134675612972288), {}).


INFO:tensorflow:Assets written to: emotion_bert_model/assets


INFO:tensorflow:Assets written to: emotion_bert_model/assets


✅ Model and LabelEncoder saved successfully
